# Importações Gerais

In [4]:
from abc import ABC, abstractmethod
import pandas as pd
import requests
import json
import time
import pandas as pd
import google.generativeai as genai
import re


# Importações para modelos "locais"

In [5]:
# https://www.youtube.com/watch?v=jsCUDeg_Op4&t=185s

!pip install -q -U transformers accelerate bitsandbytes
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM, pipeline
import shutil
import torch
import gc
import time
import transformers
from transformers import BitsAndBytesConfig
from accelerate import dispatch_model


# Gateways
Os gateways são responsáveis por chamar as APIs dos LLMs. Os gateways devem receber todos os parametros que serão passados para a API ja tratados. Além disso, os gateways devem tratar as respostas para que seja retornado apenas o texto de resposta já tratado.

In [7]:

class LLMGateway(ABC):
    def __init__(self, model_name: str, prompt_template: str):
        self.model_name = model_name
        self.prompt_template = prompt_template

    @abstractmethod
    def sendRequestToAPI(self, prompt: str) -> str:
        """
        Método abstrato que envia a requisição para o LLM e retorna a resposta.
        """
        pass

class GeminiGateway(LLMGateway):
    def __init__(self, model_name: str, prompt_template: str, api_key: str):
        super().__init__(model_name, prompt_template)
        self.api_key = api_key

    def sendRequestToAPI(self, prompt: str) -> str:
        generation_config = {
            "candidate_count": 1,
        }
        safety_settings = {
            "HARASSMENT": "BLOCK_NONE",
            "HATE": "BLOCK_NONE",
            "SEXUAL": "BLOCK_NONE",
            "DANGEROUS": "BLOCK_NONE",
        }

        nome_modelo = self.model_name.replace("google/", "models/")
        genai.configure(api_key=self.api_key)

        model = genai.GenerativeModel(
            model_name=nome_modelo,
            generation_config=generation_config,
            safety_settings=safety_settings
        )

        try:
            resposta_completa = model.generate_content(prompt)
            resposta = resposta_completa.text
        except Exception as e:
            print("Erro inesperado:", e)
            return ""

        time.sleep(30)  # Pausa para respeitar limites da API, se necessário
        return resposta


class HuggingFaceGateway(LLMGateway):
    def __init__(self, model_name: str, prompt_template: str, quantization_config):
        super().__init__(model_name, prompt_template)
        self.quantization_config = quantization_config

    def sendRequestToAPI(self) -> str: ##load do modelo
        
        if self.model_name == "google/flan-t5-small":
            tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            model = AutoModelForSeq2SeqLM.from_pretrained(
                self.model_name,
                quantization_config= self.quantization_config,
                device_map="auto",  # Automatically map the model to available devices (cpu or gpu)
            )
        else:
            tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                quantization_config= self.quantization_config,
                device_map="cuda",  # Automatically map the model to available gpu
            )
    
        # Create the pipeline
        pipe = pipeline(
           "text2text-generation" if model == "google/flan-t5-small" else "text-generation",
            model=model,
            tokenizer=tokenizer,
        )

        return pipe, model
        


# Conectores
Os conectores são responsáveis por receber os parâmetros para a realização dos testes e trata-los para que possam ser passados para a api do LLM. Nos conectores são definidas configurações padrão como chaves de API, temperatura, quantização dos modelos, etc.


In [ ]:
class LLMConnector(ABC):
    def __init__(self, model_name: str, prompt_template: str):
        self.model_name = model_name
        self.prompt_template = prompt_template

    @abstractmethod
    def callGateway(self, prompt: str) -> str:
        """
        Método abstrato que envia a requisição para o LLM e retorna a resposta.
        """
        pass

class GeminiConnector(LLMConnector):
    def __init__(self, model_name: str, prompt_template: str):
         super().__init__(model_name, prompt_template)
         self.api_key = '[GEMINI_API_KEY]'
    
    def callGateway(self):
        gateway = GeminiGateway(model_name="google/gemini-2.0-flash", prompt_template="", api_key= self.api_key); 
        response = gateway.sendRequestToAPI(self.prompt_template); 
        return response;

class HuggingFaceConnector(LLMConnector):
    def __init__(self, model_name: str, prompt_template: str):
        super().__init__(model_name, prompt_template)
        self.api_key = '[HUGGING_FACE_TOKEN]'
        self.quantization = False
        self.pipe = None
        self.model = None

        
        from huggingface_hub import login
        login(token=self.api_key)
        print('Loged on hugging face')

    def callGateway(self):
        if self.pipe is not None:
            response = self.pipe(self.prompt_template, max_length=999, num_return_sequences=1, truncation=True)
            return response[0]['generated_text'].replace(self.prompt_template,"").strip()
        # Enable quantization using bitsandbytes (e.g., 4-bit)
        if self.quantization:
          quantization_config = BitsAndBytesConfig(
              load_in_4bit=True,  # Enable 4-bit quantization
              bnb_4bit_compute_dtype=torch.float16,  # Use float16 for computations
              bnb_4bit_use_double_quant=True,  # Use double quantization for memory efficiency
              bnb_4bit_quant_type="nf4"  # Use normalized float4 for better accuracy
          )
        else:
          quantization_config = BitsAndBytesConfig(
              load_in_4bit=False,  # Enable 4-bit quantization
              bnb_4bit_compute_dtype=torch.float32,  # Use float16 for computations
              bnb_4bit_use_double_quant=False,  # Use double quantization for memory efficiency
              bnb_4bit_quant_type="nf4"  # Use normalized float4 for better accuracy
          )
        gateway = HuggingFaceGateway(self.model_name, self.prompt_template, quantization_config)
        self.pipe, self.model = gateway.sendRequestToAPI()
        response = self.pipe(self.prompt_template, max_length=999, num_return_sequences=1, truncation=True)
        return response[0]['generated_text'].replace(self.prompt_template,"").strip()
        # self.unloadModel(model, pipe, prompt_template, quantization_config)

    # def unloadModel(model, pipe):
    #     return 


# Resolvedor de provas
Essa  é a classe que resolve as provas da OAB

In [9]:
class TestMaker:
    def __init__(self, model_name: str, prompt_template: str, oabExamDf: pd.DataFrame, hint: bool):
        self.model_name = model_name
        self.prompt_template = prompt_template
        self.oabExamDf = oabExamDf  # agora é um DataFrame
        self.hint = hint
        self.huggingFaceConnector = None  # <- aqui armazenamos o conector


    def callLLMConnector(self, prompt):
        if(self.model_name == 'google/gemini-2.0-flash'):
            connector = GeminiConnector(
            model_name= self.model_name,
            prompt_template=prompt,
            )

            return connector.callGateway()

        else:
            if(self.huggingFaceConnector is None):
                self.huggingFaceConnector = HuggingFaceConnector(
                model_name=self.model_name,
                prompt_template=prompt  
            )
                
            self.huggingFaceConnector.prompt_template = prompt
            return self.huggingFaceConnector.callGateway()

    def buildPrompt(self, question: str, legalTools: str) -> str:
        prompt = self.prompt_template.format(
            question = question,
            legalTools = f"Utilize exclusivamente o ferramental juridico fornecido para responder a questão. Não útilize outras leis além das fornecidas:  {legalTools}" if self.hint else ""
        )
        return prompt

    def getChosenAlternative(self, response):
        match = re.search(r"Alternativa:\s*\*{0,2}([A-Ea-e])\*{0,2}", response, re.IGNORECASE)
        if match:
            return match.group(1).upper()
        else: 
            print('Erro ao separar alternativa');
        

    def makeTest(self):
        connectorParams = {}
        resolvedTest = pd.DataFrame()
        llmAnswerList = []
        
        for index, question in enumerate(self.oabExamDf['question'], start=0):
            print(f'Resolvendo questão {index + 1}')
            legalTools = self.oabExamDf['legalPrinciples'].iloc[index]
            prompt = self.buildPrompt(question,legalTools)
            start_time = time.time()
            llmResponse = self.callLLMConnector(prompt)
            end_time = time.time()
            alternative = self.getChosenAlternative(llmResponse)

            row = pd.DataFrame([{
                'full_model_name': self.model_name,
                'model_family': self.model_name.split('/')[0],
                'llm_chosen_answer': alternative,
                'is_correct': True if alternative == self.oabExamDf['answers'].iloc[index] else False,
                'prompt': prompt,
                'llm_response': llmResponse,
                'token_size_question': len(question.split()),
                'token_size_response':len(llmResponse.split()),
                'elapsed_time_sec': end_time - start_time,
                'hint': self.hint
            }])
            
            resolvedTest = pd.concat([resolvedTest, row], ignore_index=True)

        return resolvedTest

# Gerar csv de provas

In [11]:
PROMPT_TEMPLATE = """ Responda a questão da OAB abaixo apresentando a alternativa correta e a justificativa da escolha. 
-Questão:
{question}

{legalTools}

Retorne a resposta **EXCLUSIVAMENTE** no seguinte formato:
'Alternativa: LETRA, Justificativa: Explicação da escolha'
"""

In [12]:
MODEL = 'google/gemini-2.0-flash'
HINT = False
FILE_PATH = '/kaggle/input/oab-oficial-datasets/OAB-38.csv'

df = pd.read_csv(FILE_PATH)

# Instanciando o testMaker
tm = TestMaker(
    model_name= MODEL,
    prompt_template= PROMPT_TEMPLATE,
    oabExamDf=df,
    hint= HINT,
)

# Executando o teste
response_df = tm.makeTest()
df_final = pd.concat([df.reset_index(drop=True), response_df.reset_index(drop=True)], axis=1)

oabVersion = FILE_PATH.split('/')[4].split('.')[0].lower()
dfName = MODEL.replace('/', '_') + '_' + oabVersion + '_' + 'portuguese' + '_' + ('hint' if HINT else 'noHint')

df_final.to_csv(f"/kaggle/working/{dfName.lower()}.csv", index=False)
df_final.to_excel(f"/kaggle/working/{dfName.lower()}.xlsx", index=False, engine="openpyxl")


Resolvendo questão 1
Resolvendo questão 2
Resolvendo questão 3
Resolvendo questão 4
Resolvendo questão 5
Resolvendo questão 6
Resolvendo questão 7
Resolvendo questão 8
Resolvendo questão 9
Resolvendo questão 10
Resolvendo questão 11
Resolvendo questão 12
Resolvendo questão 13
Resolvendo questão 14
Resolvendo questão 15
Resolvendo questão 16
Resolvendo questão 17
Resolvendo questão 18
Resolvendo questão 19
Resolvendo questão 20
Resolvendo questão 21
Resolvendo questão 22
Resolvendo questão 23
Resolvendo questão 24
Resolvendo questão 25
Resolvendo questão 26
Resolvendo questão 27
Resolvendo questão 28
Resolvendo questão 29
Resolvendo questão 30
Resolvendo questão 31
Resolvendo questão 32
Resolvendo questão 33
Resolvendo questão 34
Resolvendo questão 35
Resolvendo questão 36
Resolvendo questão 37
Resolvendo questão 38
Resolvendo questão 39
Resolvendo questão 40
Resolvendo questão 41
Resolvendo questão 42
Resolvendo questão 43
Resolvendo questão 44
Resolvendo questão 45
Resolvendo questão 

In [19]:
df_final[df_final['llm_chosen_answer'].isna()]


,question,comment,legalPrinciples,answers,canceledQuestion?,full_model_name,model_family,llm_chosen_answer,is_correct,prompt,llm_response,token_size_question,token_size_response,elapsed_time_sec,hint
